# Cyprus - Weather Forecast



## 1.Setup

In [ ]:
from IPython.display import clear_output

In [ ]:
!pip install openmeteo-requests
!pip install requests-cache retry-requests numpy pandas

clear_output()

In [ ]:
import openmeteo_requests
import requests_cache
import pandas as pd
from retry_requests import retry

In [ ]:
# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

## 2.Get wind forecast

In [ ]:
def get_cy_wind_forecast(lati, longi, suffix):
  url = "https://api.open-meteo.com/v1/forecast"
  params = {
	  "latitude": lati,
	  "longitude": longi,
	  "hourly": ["wind_speed_10m", "wind_direction_10m", "wind_gusts_10m"],
	  "timezone": "Europe/Moscow",
	  "forecast_days": 2
  }
  responses = openmeteo.weather_api(url, params=params)

  # Process first location. Add a for-loop for multiple locations or weather models
  response = responses[0]
  # print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
  # print(f"Elevation {response.Elevation()} m asl")
  # print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
  # print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

  # Process hourly data. The order of variables needs to be the same as requested.
  hourly = response.Hourly()
  hourly_wind_speed_10m = hourly.Variables(0).ValuesAsNumpy()
  hourly_wind_direction_10m = hourly.Variables(1).ValuesAsNumpy()
  hourly_wind_gusts_10m = hourly.Variables(2).ValuesAsNumpy()

  hourly_data = {"date": pd.date_range(
	  start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	  end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	  freq = pd.Timedelta(seconds = hourly.Interval()),
	  inclusive = "left"
  )}

  # hourly_data['date'] = hourly_data['date'].dt.tz_convert('Europe/Athens')
  hourly_data["wind_speed_10m"] = hourly_wind_speed_10m
  hourly_data["wind_direction_10m"] = hourly_wind_direction_10m
  hourly_data["wind_gusts_10m"] = hourly_wind_gusts_10m

  hourly_dataframe = pd.DataFrame(data = hourly_data)
  hourly_dataframe['date'] = hourly_dataframe['date'].dt.tz_convert('Europe/Athens')

  # Adding suffix to columns 2 to 4
  for col in hourly_dataframe.columns[1:]:  # Columns 2 to end
    hourly_dataframe.rename(columns={col: col + suffix}, inplace=True)

  return hourly_dataframe

In [ ]:
df_aa = get_cy_wind_forecast(34.96, 33.49, '_aa') # ayia anna windfarm

In [ ]:
df_aa.dtypes

date                     datetime64[ns, Europe/Athens]
wind_speed_10m_aa                              float32
wind_direction_10m_aa                          float32
wind_gusts_10m_aa                              float32
dtype: object

## 3.Get temperature forecast (Nicosia)

In [ ]:
def get_cy_temp_forecast(lati, longi, suffix):

  url = "https://api.open-meteo.com/v1/forecast"

  params = {
	  "latitude": lati,
	  "longitude": longi,
	  "hourly": "apparent_temperature",
	  "timezone": "Europe/Moscow",
	  "forecast_days": 2
  }

  responses = openmeteo.weather_api(url, params=params)

  # Process first location. Add a for-loop for multiple locations or weather models
  response = responses[0]

  # Process hourly data. The order of variables needs to be the same as requested.
  hourly = response.Hourly()
  hourly_apparent_temperature = hourly.Variables(0).ValuesAsNumpy()

  hourly_data = {"date": pd.date_range(
	  start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	  end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	  freq = pd.Timedelta(seconds = hourly.Interval()),
	  inclusive = "left"
  )}

  hourly_data["apparent_temperature"] = hourly_apparent_temperature

  hourly_dataframe = pd.DataFrame(data = hourly_data)

  # Adding suffix
  for col in hourly_dataframe.columns[1:]:  # Columns 2 to end
    hourly_dataframe.rename(columns={col: col + suffix}, inplace=True)

  return hourly_dataframe

## 4.Combine for 24h forecast

In [ ]:
def get_cy_weather_forecast():

  # Alexigiros windfarm -> https://www.thewindpower.net/windfarm_en_15412_alexigros.php
  df_ag = get_cy_wind_forecast(34.8694, 33.5168, '_ag')
  # Ayia Anna -> https://www.thewindpower.net/windfarm_en_15926_ayia-anna.php
  df_aa = get_cy_wind_forecast(34.9609, 33.4916, '_aa')
  # Orites windfarm -> https://www.thewindpower.net/windfarm_en_17098_orites.php
  df_or = get_cy_wind_forecast(34.7394, 32.6574, "_or")
  # Nicosia apparent temperature
  df_nic = get_cy_temp_forecast(35.19, 33.38, "_nic") # nicosia temperature

  df_combined = df_ag.merge(df_aa, on='date', how='inner').merge(df_or, on='date', how='inner').merge(df_nic, on='date', how='inner')

  # Create a naive datetime64 object (without time zone)
  current_datetime_utc = pd.Timestamp.utcnow().to_datetime64()

  # Convert to a Timestamp and add a time zone (for example, UTC)
  timestamp_with_tz = pd.Timestamp(current_datetime_utc).tz_localize('UTC').tz_convert('Europe/Athens')

  # Filter for future forecast
  df_combined = df_combined[df_combined.date>timestamp_with_tz].reset_index(drop = True)
  df_combined = df_combined.head(24)

  # df_combined['index'] = 1
  df_combined['fc_h'] = range(1, len(df_combined) + 1)

  df_combined = df_combined.drop(columns=['date']).melt(id_vars=['fc_h'], var_name='variable', value_name='measurement')

  df_combined['timestamp'] = timestamp_with_tz.round('S')

  return df_combined

In [ ]:
get_cy_weather_forecast()

,fc_h,variable,measurement,timestamp
0,1,wind_speed_10m_ag,19.721298,2024-06-26 15:04:22+03:00
1,2,wind_speed_10m_ag,18.057508,2024-06-26 15:04:22+03:00
2,3,wind_speed_10m_ag,13.049689,2024-06-26 15:04:22+03:00
3,4,wind_speed_10m_ag,7.100310,2024-06-26 15:04:22+03:00
4,5,wind_speed_10m_ag,1.527351,2024-06-26 15:04:22+03:00
...,...,...,...,...
235,20,apparent_temperature_nic,39.429123,2024-06-26 15:04:22+03:00
236,21,apparent_temperature_nic,42.243244,2024-06-26 15:04:22+03:00
237,22,apparent_temperature_nic,41.634148,2024-06-26 15:04:22+03:00
238,23,apparent_temperature_nic,42.886421,2024-06-26 15:04:22+03:00


In [ ]:
get_cy_weather_forecast().variable.unique()

array(['wind_speed_10m_ag', 'wind_direction_10m_ag', 'wind_gusts_10m_ag',
       'wind_speed_10m_aa', 'wind_direction_10m_aa', 'wind_gusts_10m_aa',
       'wind_speed_10m_or', 'wind_direction_10m_or', 'wind_gusts_10m_or',
       'apparent_temperature_nic'], dtype=object)

In [ ]:
# def get_cy_current_forecast(num_fc_hrs):

#   df_ag = get_weather_forecast(34.87, 33.52, '_ag') #alexigiros windfarm
#   df_aa = get_weather_forecast(34.96, 33.49, '_aa') # ayia anna windfarm
#   df_or = get_weather_forecast(34.74, 32.66, "_or") # orites windfarm

#   df_combined = df_ag.merge(df_aa, on='date', how='inner').merge(df_or, on='date', how='inner')

#   # Create a naive datetime64 object (without time zone)
#   current_datetime_utc = pd.Timestamp.utcnow().to_datetime64()

#   # Convert to a Timestamp and add a time zone (for example, UTC)
#   timestamp_with_tz = pd.Timestamp(current_datetime_utc).tz_localize('UTC').tz_convert('Europe/Athens')

#   # Filter for future forecast
#   df_combined = df_combined[df_combined.date>timestamp_with_tz].reset_index(drop = True)

#   df_combined['index'] = 1
#   df_combined['fc_h'] = range(1, len(df_combined) + 1)

#   # Pivoting the DataFrame
#   columns_to_pivot = ['index', 'fc_h', 'date']  # Adjust as per your actual column names
#   selected_columns = [col for col in df_combined.columns if col not in columns_to_pivot] #   # Selecting all columns except 'index', 'fc_h', 'date' using a for loop

#   df_combined_pivoted = df_combined.head(num_fc_hrs).pivot(index='index', columns='fc_h', values=selected_columns)

#   # Flattening the multi-index columns
#   df_combined_pivoted.columns = [f'{name}_{hour}' for name, hour in df_combined_pivoted.columns]

#   df_combined_pivoted = df_combined_pivoted.reset_index(drop = True)

#   # Create a list of columns with the desired order
#   df_combined_pivoted['timestamp'] = timestamp_with_tz
#   # Reorder the columns in the DataFrame
#   new_column_order = ['timestamp'] + [col for col in df_combined_pivoted.columns if col != 'timestamp']
#   df_combined_pivoted = df_combined_pivoted[new_column_order]

#   return df_combined_pivoted